<a href="https://colab.research.google.com/github/efatmae/Cyber_bullying_literature_data_analysis/blob/master/pre_process_reddit_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import glob

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Process the body text 
1.   Removing special characters, new lines, https, deleted commnts..etc **(Function: clean_text)**
2.   retunr the number of words for each post to e used for statistical analysis later **(Function: return_number_words_in_text_field)**



In [ ]:
def clean_text (dataFrame, text_field):
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'http(\S)+', r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'http ...', r'')

  dataFrame[text_field] = [''.join([i if ord(i) < 128 else '' for i in str(text)]) for text in dataFrame[text_field]]
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'_[\S]?',r'')

  dataFrame[text_field] = dataFrame[text_field].str.replace(r'[ ]{2, }',r' ')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'&amp;?',r'and')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'&lt;',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'&gt;',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'"',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'>',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'<',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'[',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r']',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'?',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'\n',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'\t',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'\r',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r":",r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'|',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'/',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'(',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r')',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'#',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r';',r'')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'!',r'')

  dataFrame[text_field] = dataFrame[text_field].str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
  dataFrame[text_field] = dataFrame[text_field].str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')
  
  dataFrame[text_field] = dataFrame[text_field].str.lower()
  dataFrame[text_field] = dataFrame[text_field].str.strip()

  return dataFrame[text_field]

In [ ]:
def return_number_words_in_text_field (dataFrame_text_field):
  dataFrame_text_field_length = [len(str(text).split(' ')) for text in dataFrame_text_field]
  return dataFrame_text_field_length

In [ ]:
def preprocess_text (csv_file,text_field):
  data_df = pd.read_csv(csv_file, dtype='unicode', index_col=0)
  print("length before cleaning", len(data_df))
  data_df = data_df[data_df[text_field]!= "[deleted]"]
  data_df = data_df[data_df[text_field]!= ""]
  data_df = data_df[data_df[text_field]!= " "]
  data_df["clean_text"] = clean_text (data_df, text_field)
  data_df = data_df.dropna()
  data_df = data_df.drop_duplicates(subset=["clean_text"])
  print("length after cleaning", len(data_df))
  data_df["clean_text_length"] = return_number_words_in_text_field(data_df["clean_text"])
  data_df = data_df[data_df["clean_text_length"] >= 3]
  return data_df

In [ ]:
Reddit_csv_data_folder = "/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Data_CSV_2005-2012/"
Reddit_clean_csv_data_folder = "/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_CSV_2005_2012/"

# Trial

In [ ]:
clean_reddit_df_2005_12 = preprocess_text(Reddit_csv_data_folder+"RC_2005-12_csv_parsed.csv", "body")

length before cleaning 1075
length after cleaning 970


In [ ]:
clean_reddit_df_2005_12.head(5)

,controversiality,body,score,ups,subreddit,clean_text,clean_text_length
0,0,a look at vietnam and mexico exposes the myth ...,2,2,reddit.com,a look at vietnam and mexico exposes the myth ...,13
1,0,the site states what can i use it for meeting ...,1,1,reddit.com,the site states what can i use it for meeting ...,130
2,0,jython related topics by frank wierzbicki,0,0,reddit.com,jython related topics by frank wierzbicki,6
4,0,saft is by far the best extension you could ta...,1,1,reddit.com,saft is by far the best extension you could ta...,13
6,0,how to take panoramic shots simply,1,1,reddit.com,how to take panoramic shots simply,6


In [ ]:
sum_words_reddit_posts_2005_12 = sum(clean_reddit_df_2005_12["clean_text_length"])
no_reddit_posts_2005_12 = len(clean_reddit_df_2005_12)
clean_reddit_df_2005_12.to_csv("RC_2005-12_csv_parsed.csv",index=False)

# End of trial




In [ ]:
all_files = glob.glob("/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Data_CSV_2005-2012" + "/*")
data_stats_list = []
for filepath in all_files:
  filename = filepath.split('/')[-1]
  print(filename)
  clean_data_df = preprocess_text(filepath, "body")
  sum_words_reddit_posts= sum(clean_data_df["clean_text_length"])
  no_reddit_posts= len(clean_data_df)
  clean_data_df.to_csv(Reddit_clean_csv_data_folder+filename,index=False)
  data_stat = {}
  data_stat["file_name"] = filename
  data_stat["words_length_per_post"] = sum_words_reddit_posts
  data_stat["no_posts"] = len(clean_data_df)
  data_stats_list.append(data_stat)


RC_2012-12_csv_parsed_4.csv
length before cleaning 4259721
length after cleaning 2577719
RC_2012-12_csv_parsed_5.csv
length before cleaning 3239226
length after cleaning 2602428
RC_2012-12_csv_parsed_6.csv
length before cleaning 4230242
length after cleaning 2596545
RC_2012-12_csv_parsed_7.csv
length before cleaning 3693071
length after cleaning 2610901
RC_2012-12_csv_parsed_8.csv
length before cleaning 3892734
length after cleaning 2598769
RC_2012-12_csv_parsed_9.csv
length before cleaning 3395296
length after cleaning 2612796


# Data Statistics

In [3]:
Reddit_clean_csv_data_folder = "/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_CSV_2005_2012/"

In [4]:
df = pd.read_csv(Reddit_clean_csv_data_folder+"RC_2005-12_csv_parsed.csv")

In [5]:
df.columns

Index(['controversiality', 'body', 'score', 'ups', 'subreddit', 'clean_text',
       'clean_text_length'],
      dtype='object')

In [ ]:
all_files = glob.glob("/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_CSV_2005_2012" + "/*")
data_stats_list = []
for filepath in all_files:
  filename = filepath.split('/')[-1]
  print(filename)
  clean_data_df = pd.read_csv(Reddit_clean_csv_data_folder+filename)
  sum_words_reddit_posts= sum(clean_data_df["clean_text_length"])
  no_reddit_posts= len(clean_data_df)
  data_stat = {}
  data_stat["file_name"] = filename
  data_stat["words_length_per_post"] = sum_words_reddit_posts
  data_stat["no_posts"] = no_reddit_posts
  data_stats_list.append(data_stat)
data_stats_df = pd.DataFrame(data_stats_list)
data_stats_df.to_csv(Reddit_clean_csv_data_folder+"Reddit_clean_2005_2012_stats.csv")

In [17]:
data_stat = pd.read_csv(Reddit_clean_csv_data_folder+"Reddit_clean_2005_2012_stats.csv", index_col=False)

In [27]:
data_stat ["average_no_words"] = data_stat["words_length_per_post"] / data_stat["no_posts"]

In [28]:
data_stat

,Unnamed: 0,file_name,words_length_per_post,no_posts,average_no_words
0,0,RC_2005-12_csv_parsed.csv,58683,915,64.134426
1,1,RC_2006-01_csv_parsed.csv,165590,3018,54.867462
2,2,RC_2006-02_csv_parsed.csv,433376,7285,59.488813
3,3,RC_2006-03_csv_parsed.csv,575272,11061,52.009041
4,4,RC_2006-04_csv_parsed.csv,969631,15805,61.349636
...,...,...,...,...,...
180,180,RC_2012-12_csv_parsed_5.csv,101594098,2542043,39.965531
181,181,RC_2012-12_csv_parsed_6.csv,101192582,2536493,39.894682
182,182,RC_2012-12_csv_parsed_7.csv,103582223,2549989,40.620655
183,183,RC_2012-12_csv_parsed_8.csv,97977331,2536691,38.624070


In [21]:
Sum_all_words_in_all_posts = sum(data_stat["words_length_per_post"])
Sum_all_posts = sum(data_stat["no_posts"])
print("Average number of words csv file", Sum_all_words_in_all_posts / Sum_all_posts)

Average number of words csv file 41.095684180357736


In [30]:
print("The total number of posts in Millions", Sum_all_posts/1000000)

The total number of posts in Millions 369.401419


# Save posts into text files 

In [58]:
def write_csv_files_to_text_file(data, file_name):
    text_file = open(file_name,'w')
    for i in data:
        text_file.write(i+"\n\n")
    text_file.close()    

In [44]:
data_2005_2012 = pd.read_csv(Reddit_clean_csv_data_folder+"RC_2005-12_csv_parsed.csv", index_col=False)

In [59]:
write_csv_files_to_text_file(data_2005_2012["body"],"asdas")

In [60]:
! head 5 asdas

head: cannot open '5' for reading: No such file or directory
==> asdas <==
a look at vietnam and mexico exposes the myth of market liberalisation .

the site states what can i use it for meeting notes , reports , technical specs sign - up sheets , proposals and much more ..., just like any other new breeed of sites that want us to store everything we have on the web . and they even guarantee multiple levels of security and encryption etc . but what prevents these web site operators fom accessing andor stealing meeting notes , reports , technical specs sign - up sheets , proposals and much more , for competitive or personal gains ... i am pretty sure that most of them are honest , but what ' s there to prevent me from setting up a good useful site and stealing all your data call me paranoid - i am .

jython related topics by frank wierzbicki

saft is by far the best extension you could tak onto your safari

how to take panoramic shots simply



In [ ]:
Text_files_folder = "/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_text_2005_2012/"
all_files = glob.glob("/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_CSV_2005_2012" + "/*")
data_stats_list = []
for filepath in all_files:
  filename = filepath.split('/')[-1]
  filename = filename.split('.')[0]
  print(filename)
  clean_data_df = pd.read_csv(filepath)
  text_file_path = Text_files_folder+filename
  print("Text file: ",text_file_path)
  write_csv_files_to_text_file(clean_data_df["clean_text"],text_file_path)
  #!head -n 5 $text_file_path
  #!tail -n 5 $text_file_path

In [ ]:
#Compress the csv folder t osave disk space
!tar -czvf "/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_CSV_2005_2012.tar.gz"  "/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_text_2005_2012"

In [16]:
all_files = glob.glob("/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_text_2005_2012" + "/*")
for file in all_files:
  print("File Nme:" + file)
  with open(file) as myfile:
    head = [next(myfile) for x in range(5)]
    print(head)
  print("---------------------------------------------")
  #!tail -n 5 $file

File Nme:/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_text_2005_2012/RC_2006-03_csv_parsed
["it ' s not garbage , its fear mongering .\n", '\n', "actually , it ' s nice to have a new idea too , especially when you say stuff about how your thing is gonna do for the web what the mac did for the personal computer . if there was any evidence of that kind of thing to be smelled anywhere , i think you ' d find a little more willingness to wait and see on the execution .\n", '\n', "i ' ve read them , and they are not that funny . i would suggest you go to bash . org and read top 100 , those are much better .\n"]
---------------------------------------------
File Nme:/content//drive/My Drive/Colab_Notebooks/Reddit_data/Reddit_Clean_Data_text_2005_2012/RC_2006-04_csv_parsed
['hmm . well , apparently 100 % of 23 year olds think any kind of sex is ok . 100 % of 24 year olds think casual sex is the gateway drug to satan . 100 % of 25 year olds think casual sex is ok . hmm